In [1]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')  # Small & fast

/home/hay4hi/miniconda3/envs/testing-faiss/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from  chunking import Chunking

database_path = "/home/hay4hi/development/SW/MNXT-NAL3D/nal3d/repos/the-mad-rag/database_friends_big"
text_files = [f for f in os.listdir(database_path) if f.endswith('.txt')]
print(text_files)

database_embeddings = []
database_chunked = []

# Read the text document
for text_file in text_files:
    with open(os.path.join(database_path, text_file), 'r') as file:
        large_document = file.read()

    chunker = Chunking()

    # Step 1: Divide the text into chunks (e.g., sentences)
    #chunker.chunking_into_sentences(large_document)
    chunker.chunking_sliding_window(large_document, window_size=10, stride=1)
    database_chunked.extend(chunker.chunks)

['friends_S3_E2.txt', 'friends_S2_E1.txt', 'friends_S7_E12.txt', 'friends_S10_E9.txt', 'friends_S3_E10.txt', 'friends_S8_E20.txt', 'friends_S4_E2.txt', 'friends_S5_E9.txt', 'friends_S5_E13.txt', 'friends_S9_E19.txt', 'friends_S2_E3.txt', 'friends_S3_E7.txt', 'friends_S3_E22.txt', 'friends_S3_E15.txt', 'friends_S4_E7.txt', 'friends_S3_E11.txt', 'friends_S4_E3.txt', 'friends_S10_E12.txt', 'friends_S6_E3.txt', 'friends_S10_E4.txt', 'friends_S8_E17.txt', 'friends_S6_E7.txt', 'friends_S7_E16.txt', 'friends_S10_E14.txt', 'friends_S9_E11.txt', 'friends_S5_E4.txt', 'friends_S10_E10.txt', 'friends_S4_E4.txt', 'friends_S6_E18.txt', 'friends_S10_E6.txt', 'friends_S7_E21.txt', 'friends_S10_E1.txt', 'friends_S8_E7.txt', 'friends_S5_E20.txt', 'friends_S2_E20.txt', 'friends_S1_E2.txt', 'friends_S1_E7.txt', 'friends_S2_E11.txt', 'friends_S6_E23.txt', 'friends_S8_E19.txt', 'friends_S5_E7.txt', 'friends_S9_E12.txt', 'friends_S6_E2.txt', 'friends_S10_E16.txt', 'friends_S3_E12.txt', 'friends_S8_E14.txt', 

In [3]:
import numpy as np
database_embeddings = model.encode(database_chunked)
database_embeddings = np.array(database_embeddings).astype('float32')

In [4]:
database_embeddings.shape

(109301, 384)

In [5]:
import faiss
dimension = database_embeddings.shape[1]  # 384 for MiniLM
index = faiss.IndexFlatL2(dimension)
index.add(database_embeddings)

In [6]:
query = "what is the backpacking story ?"
query_embedding = model.encode([query]).astype('float32')

In [7]:
k = 3  # top 3 most similar sentences
D, I = index.search(query_embedding, k)

# 7. Output results
print(f"\nQuery: {query}\n")
for i, idx in enumerate(I[0]):
    print(f"Rank {i+1}: {database_chunked[idx]} (Distance: {D[0][i]:.4f})")


Query: what is the backpacking story ?

Rank 1: Y’know umm,
a few years ago I actually was backpacking across Western Europe. Kristen: Really? Ross: Uh-hmm—Wait! It gets better. Um, yeah I was in Barcelona. Kristen: I studied for a year in Barcelona. (Ross is stunned and worried.) Ross: Anyway, umm so I was um, I was hiking… Kristen: I love hiking! Ross: (whines) Oh that’s great! I was hiking along the foothills of Mount
Tibidaybo… Kristen: I think its Tibidabo. (Distance: 1.0901)
Rank 2: In the
biggest library in the university! (They start heading that way, towards a secluded
section behind the racks.) Chandler: Wow that’s actually pretty cool. Ross: (stopping quickly) Oh umm, there’s also a book here by a woman named
Wendy Bagina. (They both laugh, but stop when the hear moaning coming from the next
aisle.) What is that? Chandler: Sounds like two people are really enjoying the Dewey decimal system. (They go around the last row of bookshelves and find a couple doing what college coe